In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [72]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-02-27 22:00:00+00:00


In [73]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [74]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-02-27 16:29:23,990 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:29:24,010 INFO: Initializing external client
2025-02-27 16:29:24,011 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:29:24,797 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648


In [75]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.42s) 


In [76]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,207,0.0,2025-02-27 06:00:00+00:00
1,114,119.0,2025-02-27 06:00:00+00:00
2,47,0.0,2025-02-27 06:00:00+00:00
3,2,0.0,2025-02-27 06:00:00+00:00
4,107,33.0,2025-02-27 06:00:00+00:00
...,...,...,...
511,196,1.0,2025-02-27 22:00:00+00:00
512,95,1.0,2025-02-27 22:00:00+00:00
513,246,69.0,2025-02-27 22:00:00+00:00
514,212,0.0,2025-02-27 22:00:00+00:00


In [77]:
df['pickup_hour'].min()

Timestamp('2025-02-27 06:00:00+0000', tz='Etc/UTC')

In [78]:
df['pickup_hour'].max()

Timestamp('2025-02-27 22:00:00+0000', tz='Etc/UTC')

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516 entries, 0 to 515
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  516 non-null    int32                  
 1   predicted_demand    516 non-null    float64                
 2   pickup_hour         516 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 10.2 KB


In [80]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    # year = next_hour.year
    # month = next_hour.month
    # day = next_hour.day
    # hour = next_hour.hour

    # # Create date strings in YYYY-MM-DD format
    # current_date = f"{year}-{month:02d}-{day:02d}"
    # next_date = next_hour + timedelta(days=1)
    # next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )
    df = fg.read()
    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    # hour_str = f"{current_date} {hour:02d}:00"
    # df = df[df['pickup_hour'] == hour_str]
    df = df[df['pickup_hour'] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    # print(f"Querying for date range: {current_date} to {next_date_str}")
    # print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df)} records")
    return df

# Example usage:


In [81]:
predictions = fetch_next_hour_predictions()

2025-02-27 16:29:52,806 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:29:52,820 INFO: Initializing external client
2025-02-27 16:29:52,821 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:29:53,676 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.49s) 
Current UTC time: 2025-02-27 21:29:52.805791+00:00
Next hour: 2025-02-27 22:00:00+00:00
Found 258 records


In [82]:
now = datetime.now(timezone.utc)

In [83]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
258,47,0.0,2025-02-27 22:00:00+00:00
259,262,99.0,2025-02-27 22:00:00+00:00
260,163,107.0,2025-02-27 22:00:00+00:00
261,191,0.0,2025-02-27 22:00:00+00:00
262,233,53.0,2025-02-27 22:00:00+00:00
...,...,...,...
511,196,1.0,2025-02-27 22:00:00+00:00
512,95,1.0,2025-02-27 22:00:00+00:00
513,246,69.0,2025-02-27 22:00:00+00:00
514,212,0.0,2025-02-27 22:00:00+00:00


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516 entries, 0 to 515
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  516 non-null    int32                  
 1   predicted_demand    516 non-null    float64                
 2   pickup_hour         516 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 10.2 KB


In [85]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [86]:
dt = current_date.ceil('h')


In [87]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour >= "2025-02-26 05:00:00")  
results = query.read()

2025-02-27 16:31:22,660 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:31:22,675 INFO: Initializing external client
2025-02-27 16:31:22,675 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:31:23,418 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.49s) 


In [88]:
fg.filter((fg.pickup_hour >= "2025-02-27") & (fg.pickup_hour < "2025-02-28")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.45s) 


,pickup_location_id,predicted_demand,pickup_hour
0,207,0.0,2025-02-27 06:00:00+00:00
1,114,119.0,2025-02-27 06:00:00+00:00
2,47,0.0,2025-02-27 06:00:00+00:00
3,2,0.0,2025-02-27 06:00:00+00:00
4,107,33.0,2025-02-27 06:00:00+00:00
...,...,...,...
253,32,0.0,2025-02-27 06:00:00+00:00
254,184,0.0,2025-02-27 06:00:00+00:00
255,165,1.0,2025-02-27 06:00:00+00:00
256,56,0.0,2025-02-27 06:00:00+00:00


In [89]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,207,0.0,2025-02-27 06:00:00+00:00
1,114,119.0,2025-02-27 06:00:00+00:00
2,47,0.0,2025-02-27 06:00:00+00:00
3,2,0.0,2025-02-27 06:00:00+00:00
4,107,33.0,2025-02-27 06:00:00+00:00
...,...,...,...
511,196,1.0,2025-02-27 22:00:00+00:00
512,95,1.0,2025-02-27 22:00:00+00:00
513,246,69.0,2025-02-27 22:00:00+00:00
514,212,0.0,2025-02-27 22:00:00+00:00


In [90]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [91]:
predictions = fetch_next_hour_predictions()

2025-02-27 16:32:04,154 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:32:04,166 INFO: Initializing external client
2025-02-27 16:32:04,167 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:32:05,146 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648
Current UTC time: 2025-02-27 21:32:04.153864+00:00
Next hour: 2025-02-27 22:00:00+00:00
Found 258 records


In [92]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-02-27 16:32:11,338 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:32:11,342 INFO: Initializing external client
2025-02-27 16:32:11,342 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:32:12,187 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648


In [93]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.44s) 


In [94]:
df["pickup_hour"].max()

Timestamp('2025-02-27 22:00:00+0000', tz='Etc/UTC')

In [95]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-02-27 22:00:00+00:00


In [96]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
258,47,0.0,2025-02-27 22:00:00+00:00
259,262,99.0,2025-02-27 22:00:00+00:00
260,163,107.0,2025-02-27 22:00:00+00:00
261,191,0.0,2025-02-27 22:00:00+00:00
262,233,53.0,2025-02-27 22:00:00+00:00
...,...,...,...
511,196,1.0,2025-02-27 22:00:00+00:00
512,95,1.0,2025-02-27 22:00:00+00:00
513,246,69.0,2025-02-27 22:00:00+00:00
514,212,0.0,2025-02-27 22:00:00+00:00
